In [11]:
import numpy as np
import pandas as pd
import time, warnings
import datetime as dt
warnings.filterwarnings("ignore")

df=pd.read_excel(r'C:\Users\gaura\PycharmProjects\pythonProject\Data.xlsx')
df.head()

,Customer_Name,Product_ID,Product_Name,Feature_ID,Feature_Name,Invoice_Date,Cost,City,Contact_Number,Partner_Name
0,Winifred,4,Software 5,2,Feature C,2015-04-28 01:32:43,1730,Kolkata,9185369916,Simone
1,Bert,3,Software 4,3,Feature D,2017-07-13 08:02:47,1214,Kolkata,9477822936,Nathaniel
2,Matthew,0,Software 1,1,Feature B,2016-03-03 11:44:45,1419,Bangalore,9811153392,William
3,Samuel,4,Software 5,0,Feature A,2017-01-08 21:36:32,942,Chennai,9359971220,Edward
4,Jennifer,4,Software 5,7,Feature H,2017-12-27 05:39:30,1488,Chennai,9176571058,Raymond


In [12]:
df['date'] = pd.DatetimeIndex(df['Invoice_Date']).date
recency_df = df.groupby(by='Customer_Name', as_index=False)['date'].max()
recency_df.columns = ['Customer_Name','LastPurchaseDate']
recent_date=recency_df.LastPurchaseDate.max()
print(recent_date)

recency_df['Recency'] = recency_df['LastPurchaseDate'].apply(lambda x: (recent_date - x).days)
recency_df.head()

2017-12-30


,Customer_Name,LastPurchaseDate,Recency
0,Aaron,2016-11-19,406
1,Adam,2016-05-27,582
2,Adrienne,2016-12-18,377
3,Agnes,2017-09-05,116
4,Alaina,2015-08-14,869


In [13]:
df1= df
df1.drop_duplicates(subset=['Product_ID', 'Customer_Name'], keep="first", inplace=True)

frequency_df = df1.groupby(by=['Customer_Name'], as_index=False)['Product_ID'].count()
frequency_df.columns = ['Customer_Name','Frequency']
frequency_df.head()


,Customer_Name,Frequency
0,Aaron,1
1,Adam,1
2,Adrienne,1
3,Agnes,1
4,Alaina,1


In [14]:
df['TotalCost'] = df['Cost']
monetary_df = df.groupby(by='Customer_Name',as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['Customer_Name','Monetary']
monetary_df.head()

,Customer_Name,Monetary
0,Aaron,844
1,Adam,887
2,Adrienne,1436
3,Agnes,909
4,Alaina,1836


In [17]:
temp_df = recency_df.merge(frequency_df,on='Customer_Name')
temp_df.head()
rfm_df = temp_df.merge(monetary_df,on='Customer_Name')
rfm_df.set_index('Customer_Name',inplace=True)
rfm_df.head()

,LastPurchaseDate,Recency,Frequency,Monetary
Customer_Name,,,,
Aaron,2016-11-19,406,1,844
Adam,2016-05-27,582,1,887
Adrienne,2016-12-18,377,1,1436
Agnes,2017-09-05,116,1,909
Alaina,2015-08-14,869,1,1836


In [18]:
rfm_df['R_rank'] = rfm_df['Recency'].rank( ascending=False)
rfm_df['F_rank'] = rfm_df['Frequency'].rank(ascending=True)
rfm_df['M_rank'] = rfm_df['Monetary'].rank(ascending=True)
rfm_df.head()

rfm_df['R_rank_norm']=(rfm_df['R_rank']/rfm_df['R_rank'].max())*100
rfm_df['F_rank_norm']=(rfm_df['F_rank']/rfm_df['F_rank'].max())*100
rfm_df['M_rank_norm']=(rfm_df['F_rank']/rfm_df['M_rank'].max())*100
rfm_df.head()

rfm_df['RFM_Score']=0.15*rfm_df['R_rank_norm']+0.28*rfm_df['F_rank_norm']+0.57*rfm_df['M_rank_norm']
rfm_df=rfm_df.round(0)
rfm_df.head()

,LastPurchaseDate,Recency,Frequency,Monetary,R_rank,F_rank,M_rank,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score
Customer_Name,,,,,,,,,,,
Aaron,2016-11-19,406,1,844,280.0,183.0,10.0,54.0,35.0,35.0,38.0
Adam,2016-05-27,582,1,887,184.0,183.0,30.0,35.0,35.0,35.0,35.0
Adrienne,2016-12-18,377,1,1436,290.0,183.0,196.0,56.0,35.0,35.0,38.0
Agnes,2017-09-05,116,1,909,440.0,183.0,36.0,84.0,35.0,35.0,43.0
Alaina,2015-08-14,869,1,1836,82.0,183.0,320.0,16.0,35.0,35.0,32.0


In [19]:
rfm_df["Customer_segment"]=np.where(rfm_df['RFM_Score'] < 40 ,"Low Value Customer",(np.where(rfm_df['RFM_Score'] < 60 , "Medium value Customer" , (np.where(rfm_df['RFM_Score'] < 80 , "High value Customer" ,"Very High Value Customer")))))
rfm_df.head()

,LastPurchaseDate,Recency,Frequency,Monetary,R_rank,F_rank,M_rank,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score,Customer_segment
Customer_Name,,,,,,,,,,,,
Aaron,2016-11-19,406,1,844,280.0,183.0,10.0,54.0,35.0,35.0,38.0,Low Value Customer
Adam,2016-05-27,582,1,887,184.0,183.0,30.0,35.0,35.0,35.0,35.0,Low Value Customer
Adrienne,2016-12-18,377,1,1436,290.0,183.0,196.0,56.0,35.0,35.0,38.0,Low Value Customer
Agnes,2017-09-05,116,1,909,440.0,183.0,36.0,84.0,35.0,35.0,43.0,Medium value Customer
Alaina,2015-08-14,869,1,1836,82.0,183.0,320.0,16.0,35.0,35.0,32.0,Low Value Customer


In [20]:
rfm_df.to_excel("RFM Model.xlsx")